In [8]:
import cv2
import os
import matplotlib.pyplot as plt

folder_path = ".\\data_for_yolo\\data\\train\\images"
labels_path = ".\\data_for_yolo\\data\\train\\labels"

image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') and ('d1' in f or 'd2' in f)]

print(len(image_files))

for i in range(1,7):
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') and f'tom{i}' in f and ('d1' in f or 'd2' in f)]
    print(f"Tomato Farm Number {i}: {len(image_files)}")

9093
Tomato Farm Number 1: 2452
Tomato Farm Number 2: 1724
Tomato Farm Number 3: 2311
Tomato Farm Number 4: 830
Tomato Farm Number 5: 970
Tomato Farm Number 6: 806


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

folder_path = ".\\data_for_yolo\\data\\train\\images"
labels_path = ".\\data_for_yolo\\data\\train\\labels"

def parse_yolo_coordinates(yolo_str):
    parts = yolo_str.split()
    return int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])

def change_class_id(yolo_str, new_class_id):
    parts = yolo_str.split()
    parts[0] = str(new_class_id)
    return ' '.join(parts)

image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') and ('d1' in f or 'd2' in f)]

resume_file = 'V001_tom1_43_085_d2_11_20211031_18_06072546_38102607.png'
if resume_file in image_files:
    idx = image_files.index(resume_file)
    image_files = image_files[idx:]

total_files = len(image_files)
farm_counts = {f"tom{i+1}": len([f for f in image_files if f"tom{i+1}" in f]) for i in range(6)}
previous_farms_counts = 0

for i, img_name in enumerate(image_files, start=1):
    farm_name = img_name.split('_')[1]  # assuming 'tom1' always at this position

    img_path = os.path.join(folder_path, img_name)
    txt_path = os.path.join(labels_path, img_name.replace('.png', '.txt'))

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    img_width, img_height = 1080, 1920

    with open(txt_path, 'r') as file:
        yolo_strs = file.readlines()

    for idx, yolo_str in enumerate(yolo_strs):
        fig, ax = plt.subplots(1, figsize=(12, 12))
        ax.imshow(image)

        class_id, x, y, w, h = parse_yolo_coordinates(yolo_str)
        x, y, w, h = x * img_width, y * img_height, w * img_width, h * img_height
        bbox = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor='r', facecolor='none')

        ax.add_patch(bbox)

        plt.pause(0.01)
        plt.show()

        current_index = image_files.index(img_name)
        current_farm_count = current_index - previous_farms_counts

        print(f"Processing image {i} of {total_files}, farm {farm_name} ({current_farm_count + 1} of {farm_counts[farm_name]})")
        
        answer = input(f"Do you want to change class id of object {idx} in image {img_name}? (y/n): ")
        if answer.lower() == 'y':
            yolo_strs[idx] = change_class_id(yolo_strs[idx], '1') + "\n"

        plt.close()

    if i < len(image_files) and farm_name not in image_files[i]:
        previous_farms_counts += farm_counts[farm_name]

    with open(txt_path, 'w') as file:
        file.writelines(yolo_strs)

print("Finished")